## SQL Translate

In [1]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.9 MB/s eta 0:00:00


In [2]:
import openai
api_key = "sk-...................................................Qa"
openai.api_key = api_key

In [3]:
prompt = """### Microsoft SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Sales_amount(id, employee_id, amount, date)
#
### A query to list the names of employees whose total sales in the last 3 months have been more than 10 thousand EURO
SELECT"""

In [4]:
prompt

'### Microsoft SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Sales_amount(id, employee_id, amount, date)\n#\n### A query to list the names of employees whose total sales in the last 3 months have been more than 10 thousand EURO\nSELECT'

In [5]:
import openai

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0,
  max_tokens=150,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#", ";"]
)

# GPT modellerinde temperature parametresi, çıktının rastgeleliğini/kalitesini kontrol eder. 0 ile 2 arasında ayarlanır, 0 en tahmin edilebilir/en kaliteli
# ve 2 en rastgele/en kalitesiz çıktıyı döndürür. 
# Temperature 0 olduğunda, GPT her seferinde en yüksek olası yanıtı seçer. Temperature parametresi 2 olarak ayarlandığında, çıktının rastgeleliği artar ve çıktı kalitesi çok düşer.
# Ancak temperature değeri MAX 0.8 ve civarı değerler tavsiye edilir.
# temperature örnek:
# ""Kahramanımız elindeki kılıcı ile ejderhayı ....."" cümlesinde takip eden kelimelerin/tokenlerin olasılıkları %40 öldürdü, %30 yendi, %20 yaraladı, %7 korkuttu, %3 öptü olsun.
# temperature 0, 0.1 gibi çok düşük değerler olarak ayarlanırsa model takip eden kelime olarak en yüksek olasılıklı "öldürdü" kelimesini seçer.
# temperature 0.2, 0.3 gibi düşük değerler ayarlanırsa model takip eden kelime olarak en yüksek olasılıklı "öldürdü", "yendi" kelimelerinden birini seçer. 
# temperature 2 civarı değerler seçilirse model takip eden kelime olarak en düşük olasılıklı kelimeler de dahil olmak üzere olası tüm kelimelerden herhangi birini seçer.


# GPT modellerinde top_p parametresi, bir sonraki kelimenin model tarafından ne kadar çeşitli şekillerde tahmin edilebileceğini belirler. top_p=1.0 olduğunda model, 
# tüm olası kelimeleri seçebilir. top_p=0.5 olduğunda ise model, tüm olası kelimelerin ilk %50'lik kısmındaki kelimeleri seçebilir. 
# Büyük top_p (0.9 ve üzeri) değeri modelin daha fazla, küçük top_p (0.5 ve altı) değeri ise modelin daha az çeşitlilik göstermesine sağlar.
# top_p örnek:
# yukardaki örnek cümle için top_p 0.7 olarak ayarlanırsa takip eden kelimeyi, en yüksek olasılıktan başlayacak şekilde kümülatif toplamları 0.7'yi (0.4 öldürdü, 0.3 yendi) 
# geçmeyen kelimeler arasından seçer.

# top_p 0.65 olarak ayarlanırsa takip eden kelimeyi, en yüksek olasılıktan başlayacak şekilde kümülatif toplamları 0.65'i (0.4 öldürdü) geçmeyen kelimeler arasından seçer. Çünkü
# 0.4 olasılığa sahip "öldürdü" kelimesi ile 0.3 olasılığına sahip "yendi" kelimesinin kümülatif olasılık toplamı(0.7) 0.65'den büyük olduğundan sadece en büyük olasılığa sahip
# "öldürdü" kelimesini seçer "yendi" kelimesini ignore eder. 

# top_p 0.26 olarak ayarlanırsa takip eden kelimeyi, en yüksek olasılıktan başlayacak şekilde kümülatif toplamları 0.26'yı (0.2 yaraladı) geçmeyen kelimeler arasından seçer. Çünkü
# 0.2 olasılığa sahip "yaraladı" kelimesi ile 0.07 olasılığına sahip "korkuttu" kelimesinin kümülatif olasılık toplamı(0.27) 0.26'den büyük olduğundan sadece en büyük olasılığa sahip
# "yaraladı" kelimesini seçer "korkuttu" kelimesini ignore eder.

# hem top_p'nin 1 (max) hem de temperature'nin 2 (max) ayarlanması çıktının rassallığını çok fazla artıracağından alınan çıktının kalitesi çok kötü olacaktır. 
# DİKKAT, ÇOK ÖNEMLİ: 1. En olası çıktıyı elde etmek için top_p'nin her zaman 1, temperaturenin da 0 olarak ayarlanması, 
#                     2. Çıktının rassalığının artırılması istenirse top_p'nin 1'de sabit bırakılarak sadece temperature hyper_parametresi ile oynanması openai tarafından 
#                        tavsiye edilmiştir.

#  positif değerler : “Frequency penalty”, bir kelimenin kullanım sayısı arttıkça o kelimenin tekrar seçilme olasılığını düşürürken, “presence penalty” bir kelimenin daha 
# önce kullanılmış olup olmadığına bakarak tekrar seçilme şansını düşürür. Yani “presence penalty”, bir kelimenin ne kadar sıklıkla kullanıldığını dikkate almaz, sadece metinde 
# var olup olmadığına bakar. 

# negatif değerler : “Frequency penalty”, bir kelimenin kullanım sayısı arttıkça o kelimenin tekrar seçilme olasılığını artırır, “presence penalty” bir kelimenin daha 
# önce kullanılmış olup olmadığına bakarak tekrar seçilme şansını artırır.

# “Frequency penalty”, “presence penalty” -2 ile +2 arasında değer alsa da 0.1 ile 1 arasında değerlerin kullanılması tavsiye edilmiştir.

In [6]:
response

<OpenAIObject text_completion id=cmpl-7N1aggMqD9Xh36qMPtyBPdunrtt9a at 0x7ff6e24049f0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " e.name \nFROM Employee e \nINNER JOIN Sales_amount sa \nON e.id = sa.employee_id \nWHERE sa.date > DATEADD(month, -3, GETDATE()) \nGROUP BY e.name \nHAVING SUM(sa.amount) > 10000"
    }
  ],
  "created": 1685722014,
  "id": "cmpl-7N1aggMqD9Xh36qMPtyBPdunrtt9a",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 72,
    "prompt_tokens": 79,
    "total_tokens": 151
  }
}

In [7]:
response["choices"][0] # choices içerisindeki dictionary'e giriş yapıyoruz.

<OpenAIObject at 0x7ff6e175b790> JSON: {
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "text": " e.name \nFROM Employee e \nINNER JOIN Sales_amount sa \nON e.id = sa.employee_id \nWHERE sa.date > DATEADD(month, -3, GETDATE()) \nGROUP BY e.name \nHAVING SUM(sa.amount) > 10000"
}

In [8]:
print("SELECT"+response["choices"][0]["text"])

SELECT e.name 
FROM Employee e 
INNER JOIN Sales_amount sa 
ON e.id = sa.employee_id 
WHERE sa.date > DATEADD(month, -3, GETDATE()) 
GROUP BY e.name 
HAVING SUM(sa.amount) > 10000


In [9]:
prompt = """### Microsoft SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Sales_amount(id, employee_id, amount, date)
#
### A query to list the names of employees whose total sales in the last 3 months have been more than 10 thousand EURO
SELECT"""

In [10]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0,
  max_tokens=150,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["###", ";"]
)
print("SELECT"+response["choices"][0]["text"])

SELECT e.name 
FROM Employee e 
INNER JOIN Sales_amount sa 
ON e.id = sa.employee_id 
WHERE sa.date > DATEADD(month, -3, GETDATE()) 
GROUP BY e.name 
HAVING SUM(sa.amount) > 10000


#### With GPT-3.5

One-Shot Training

In [15]:
system="""You are a Microsoft SQL expert. Answer the questions asked to you in the most accurate way. Return only the query that was asked to you.
Microsoft SQL tables, with their properties:
Employee(id, name, department_id)
Department(id, name, address)
Sales_amount(id, employee_id, amount, date)"""

user1 = "write a query to list the names of the employees who live in Ankara"

assistant = """SELECT Employee.name 
FROM Employee 
JOIN Department 
ON Employee.department_id = Department.id 
WHERE Department.address = 'Ankara';"""

user2 = "write a query to list the names of employees whose total sales in the last 3 months have been more than 10 thousand EURO. just return query"

In [16]:
import openai

res = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
          {"role": "system", "content": f"{system}"},
          {"role": "user", "content": f"{user1}"},
          {"role": "assistant", "content": f"{assistant}"},
          {"role": "user", "content": f"{user2}"}
      ]
  )

In [17]:
res

<OpenAIObject chat.completion id=chatcmpl-7N1ciOCAHrjEehoaRtoWEJcv4eT1x at 0x7ff6d152e750> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "SELECT Employee.name \nFROM Employee \nJOIN Sales_amount \nON Employee.id = Sales_amount.employee_id \nWHERE Sales_amount.date >= DATEADD(month, -3, GETDATE()) \nGROUP BY Employee.id, Employee.name \nHAVING SUM(Sales_amount.amount) > 10000;",
        "role": "assistant"
      }
    }
  ],
  "created": 1685722140,
  "id": "chatcmpl-7N1ciOCAHrjEehoaRtoWEJcv4eT1x",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 57,
    "prompt_tokens": 157,
    "total_tokens": 214
  }
}

In [18]:
print(res["choices"][0]["message"]["content"])

SELECT Employee.name 
FROM Employee 
JOIN Sales_amount 
ON Employee.id = Sales_amount.employee_id 
WHERE Sales_amount.date >= DATEADD(month, -3, GETDATE()) 
GROUP BY Employee.id, Employee.name 
HAVING SUM(Sales_amount.amount) > 10000;


In [19]:
def chatgpt(system, question):

  import openai

  res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
          {"role": "system", "content": f"{system}"},
          {"role": "user", "content": f"{question}"}],
    temperature=0,
    top_p=1,
    max_tokens=500,
    presence_penalty=0,
    frequency_penalty=0,

    )

  return print(res['choices'][0]["message"]["content"])

In [21]:
system = """You are a Microsoft SQL expert. Answer the questions asked to you in the most accurate way. Return only the query that was asked to you.
Microsoft SQL tables, with their properties:
Employee(id, name, department_id)
Department(id, name, address)
Sales_amount(id, employee_id, amount, date)"""

question = "write a query to list the names of employees whose total sales in the last 3 months have been more than 10 thousand EURO. just return query"

chatgpt(system, question)

```
SELECT e.name
FROM Employee e
INNER JOIN Sales_amount sa ON e.id = sa.employee_id
WHERE sa.date >= DATEADD(month, -3, GETDATE())
GROUP BY e.id, e.name
HAVING SUM(sa.amount) > 10000
```


## Python to natural language

In [22]:
prompt ="""# Python 3 
def remove_common_prefix(x, prefix, ws_prefix): 
    x["completion"] = x["completion"].str[len(prefix) :] 
    if ws_prefix: 
        # keep the single whitespace as prefix 
        x["completion"] = " " + x["completion"] 
return x 

# Explanation of what the code does

#"""

In [23]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0,
  max_tokens=500,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#"]
)
print(response["choices"][0]["text"])

This code removes a common prefix from a given string. The parameters x, prefix, and ws_prefix are passed into the function. The x parameter is a dataframe containing a column called "completion". The prefix parameter is the string that is to be removed from the "completion" column. The ws_prefix parameter is a boolean value that determines whether or not a single whitespace should be kept as a prefix. 




In [24]:
system = "You are a python 3 expert. Answer the questions asked to you in the most accurate way."

question = """
def remove_common_prefix(x, prefix, ws_prefix): 
    x["completion"] = x["completion"].str[len(prefix) :] 
    if ws_prefix: 
        # keep the single whitespace as prefix 
        x["completion"] = " " + x["completion"] 
return x
Explanation of what the code does"""

chatgpt(system, question)

This code defines a function called `remove_common_prefix` that takes three arguments: `x`, `prefix`, and `ws_prefix`. The function removes the common prefix from the "completion" column of the input dataframe `x`. 

First, the function removes the prefix from the "completion" column by using the `str` method of pandas dataframe and slicing the string from the length of the prefix to the end of the string. 

If `ws_prefix` is True, the function adds a single whitespace as the new prefix to the "completion" column. 

Finally, the function returns the modified dataframe `x`.


## Translate programming languages

In [25]:
prompt = """##### Translate this code from R into Python3
### R
    
    set.seed(42)
    train_index <- createDataPartition(y, p = 0.9, list = FALSE)
    X_train <- X[train_index, ]
    X_test <- X[-train_index, ]
    y_train <- y[train_index]
    y_test <- y[-train_index]
    
### Python3"""

In [26]:
import openai

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0,
  max_tokens=150,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["###"]
)
print(response["choices"][0]["text"])



    np.random.seed(42)
    train_index = np.random.choice(len(y), round(0.9*len(y)), replace=False)
    X_train = X[train_index]
    X_test = X[~train_index]
    y_train = y[train_index]
    y_test = y[~train_index]


In [28]:
system = "You are a python3 and R expert. Answer the questions asked to you in the most accurate way."

question = """
set.seed(42)
train_index <- createDataPartition(y, p = 0.9, list = FALSE)
X_train <- X[train_index, ]
X_test <- X[-train_index, ]
y_train <- y[train_index]
y_test <- y[-train_index]
Translate this code from R into Python3"""

chatgpt(system, question)

import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


## Explain Code

In [ ]:
prompt= """
class Log:
    def __init__(self, path):
        dirname = os.path.dirname(path)
        os.makedirs(dirname, exist_ok=True)
        f = open(path, "a+")

        # Check that the file is newline-terminated
        size = os.path.getsize(path)
        if size > 0:
            f.seek(size - 1)
            end = f.read(1)
            if end != "\n":
                f.write("\n")
        self.f = f
        self.path = path

    def log(self, event):
        event["_event_id"] = str(uuid.uuid4())
        json.dump(event, self.f)
        self.f.write("\n")

    def state(self):
        state = {"complete": set(), "last": None}
        for line in open(self.path):
            event = json.loads(line)
            if event["type"] == "submit" and event["success"]:
                state["complete"].add(event["id"])
                state["last"] = event
        return state

\"\"\"
Here's what the above class is doing, explained in a concise way:
1."""

In [ ]:
import openai

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0,
  max_tokens=300,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\"\"\""]
)
print("1."+response["choices"][0]["text"])

1. The __init__ method creates a new file at the given path, and checks that it is newline-terminated.
2. The log method adds a new event to the log file, with a unique ID.
3. The state method reads the log file and returns a dictionary containing the set of completed tasks and the last successful event.



In [29]:
system = "You are a python3 expert. explain the python codes asked to you in the most accurate way."

question = """
class Log:
    def __init__(self, path):
        dirname = os.path.dirname(path)
        os.makedirs(dirname, exist_ok=True)
        f = open(path, "a+")

        # Check that the file is newline-terminated
        size = os.path.getsize(path)
        if size > 0:
            f.seek(size - 1)
            end = f.read(1)
            if end != "\n":
                f.write("\n")
        self.f = f
        self.path = path

    def log(self, event):
        event["_event_id"] = str(uuid.uuid4())
        json.dump(event, self.f)
        self.f.write("\n")

    def state(self):
        state = {"complete": set(), "last": None}
        for line in open(self.path):
            event = json.loads(line)
            if event["type"] == "submit" and event["success"]:
                state["complete"].add(event["id"])
                state["last"] = event
        return state
explain, point by point, what the above class does"""

chatgpt(system, question)

The above code defines a class called `Log`. Here is what it does:

1. The `__init__` method takes a `path` argument and creates a file object at that path. If the directory of the path does not exist, it creates it. It also checks if the file is newline-terminated and adds a newline character if it is not.

2. The `log` method takes an `event` argument, adds a unique `_event_id` key to the event dictionary using the `uuid.uuid4()` method, and writes the event dictionary to the file object in JSON format. It also adds a newline character after writing the event.

3. The `state` method reads the file object line by line, loads each line as a JSON object, and checks if the object has a `"type"` key with the value `"submit"` and a `"success"` key with the value `True`. If it does, it adds the `"id"` key to a set called `"complete"` and sets the `"last"` key to the current event. Finally, it returns a dictionary with the `"complete"` set and the `"last"` event.


## Python bug fixer

In [30]:
prompt ="""
##### Fix bugs in the below function
 
### Buggy Python
import Random
a = random.randint(1,12)
b = random.randint(1,12)
for i in range(10):
    question = "What is "+a+" x "+b+"? "
    answer = input(question)
    if answer = a*b
        print (Well done!)
    else:
        print("No.")
    
### Fixed Python"""

In [31]:
import openai

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0,
  max_tokens=300,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["###"]
)
print(response["choices"][0]["text"])


import random
a = random.randint(1,12)
b = random.randint(1,12)
for i in range(10):
    question = "What is "+str(a)+" x "+str(b)+"? "
    answer = int(input(question))
    if answer == a*b:
        print ("Well done!")
    else:
        print("No.")


In [32]:
system = "You are a python3 expert. fix bugs in the python codes asked to you in the most accurate way."

question = """
import Random
a = random.randint(1,12)
b = random.randint(1,12)
for i in range(10):
    question = "What is "+a+" x "+b+"? "
    answer = input(question)
    if answer = a*b
        print (Well done!)
    else:
        print("No.")"""

chatgpt(system, question)

There are several errors in the code. Here's the corrected code:

```python
import random

a = random.randint(1, 12)
b = random.randint(1, 12)

for i in range(10):
    question = "What is " + str(a) + " x " + str(b) + "? "
    answer = int(input(question))
    if answer == a * b:
        print("Well done!")
    else:
        print("No.")
```

Changes made:
- `random` module was imported with a capital R, which is incorrect. It should be `import random`.
- The variables `a` and `b` were not converted to strings before concatenating with other strings. I added `str()` to convert them to strings.
- The `input()` function returns a string, so the `answer` variable needs to be converted to an integer using `int()`.
- The `if` statement had a syntax error. The comparison operator `==` was used instead of `=`. Also, the string "Well done!" was not enclosed in quotes.
- The `else` statement was missing quotes around the string "No.".


In [33]:
system = "You are a python3 expert. fix bugs in the python codes asked to you in the most accurate way. just return fixed python codes"

question = """
import Random
a = random.randint(1,12)
b = random.randint(1,12)
for i in range(10):
    question = "What is "+a+" x "+b+"? "
    answer = input(question)
    if answer = a*b
        print (Well done!)
    else:
        print("No.")"""

chatgpt(system, question)

import random
a = random.randint(1,12)
b = random.randint(1,12)
for i in range(10):
    question = "What is "+str(a)+" x "+str(b)+"? "
    answer = int(input(question))
    if answer == a*b:
        print ("Well done!")
    else:
        print("No.")


## JavaScript to Python

In [34]:
prompt="""
#JavaScript to Python:
JavaScript: 
dogs = ["bill", "joe", "carl"]
car = []
dogs.forEach((dog) {
    car.push(dog);
});

Python:"""

In [35]:
import openai

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0,
  max_tokens=300,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)
print(response["choices"][0]["text"])


dogs = ["bill", "joe", "carl"]
car = []
for dog in dogs:
    car.append(dog)


In [36]:
system = "You are a python3 and javascript expert. Translate this code from javascript into Python3 in the most accurate way."

question = """
dogs = ["bill", "joe", "carl"]
car = []
dogs.forEach((dog) {
    car.push(dog);
});"""

chatgpt(system, question)

dogs = ["bill", "joe", "carl"]
car = []
for dog in dogs:
    car.append(dog)


## Write a Python docstring

In [39]:
prompt="""
# Python 3 
def remove_common_prefix(x, prefix, ws_prefix): 
    x["completion"] = x["completion"].str[len(prefix) :] 
    if ws_prefix: 
        # keep the single whitespace as prefix 
        x["completion"] = " " + x["completion"] 
return x 

# An elaborate, high quality docstring for the above function:
\"\"\"
"""

In [40]:
prompt

'\n# Python 3 \ndef remove_common_prefix(x, prefix, ws_prefix): \n    x["completion"] = x["completion"].str[len(prefix) :] \n    if ws_prefix: \n        # keep the single whitespace as prefix \n        x["completion"] = " " + x["completion"] \nreturn x \n\n# An elaborate, high quality docstring for the above function:\n"""\n'

In [41]:
import openai

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0,
  max_tokens=150,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#", "\"\"\""]
)
print(response["choices"][0]["text"])

remove_common_prefix(x, prefix, ws_prefix)

This function removes a common prefix from a given string.

Parameters:
    x (str): The string to remove the prefix from.
    prefix (str): The prefix to remove from the string.
    ws_prefix (bool): Whether to keep a single whitespace as the prefix.

Returns:
    x (str): The string with the prefix removed.

Raises:
    ValueError: If the prefix is not found in the string.



In [42]:
system = "You are a python3 expert. write high quality docstring for Python3 codes given to you the most accurate way."

question = """
def remove_common_prefix(x, prefix, ws_prefix): 
    x["completion"] = x["completion"].str[len(prefix) :] 
    if ws_prefix: 
        # keep the single whitespace as prefix 
        x["completion"] = " " + x["completion"] 
return x """

chatgpt(system, question)

"""
Removes a common prefix from a given string in a pandas DataFrame column.

Args:
    x (pandas.DataFrame): A pandas DataFrame containing a column named "completion" that needs to be modified.
    prefix (str): The common prefix to be removed from the "completion" column.
    ws_prefix (bool): A boolean value indicating whether to keep a single whitespace as prefix after removing the common prefix.

Returns:
    pandas.DataFrame: The modified pandas DataFrame with the common prefix removed from the "completion" column.

Example:
    >>> df = pd.DataFrame({'completion': ['apple', 'apples', 'apricot']})
    >>> df = remove_common_prefix(df, 'ap', True)
    >>> print(df)
         completion
    0       ple
    1      ples
    2     ricot
"""


## Edit Codes

In [43]:
def CodeEdit(input, instruction):

  import openai

  res = openai.Edit.create(
    model= "code-davinci-edit-001",
    input= f"{input}",
    instruction= f"{instruction}",
    temperature=0.0,
    top_p=1)
  return print(res["choices"][0]["text"])

In [45]:
CodeEdit(input=" ", instruction="Write a function in python that calculates fibonacci")

def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(9))



In [46]:
input = """def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(9))"""
CodeEdit(input=input, instruction="Add docstring")

def fibonacci(n):
    """
    Returns the nth number in the fibonacci sequence
    """
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(9))



In [47]:
input = """def fibonacci(n):
    \"\"\"
    Returns the nth number in the fibonacci sequence
    \"\"\"
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(9))"""
CodeEdit(input=input, instruction="Rename the function to fib")

def fib(n):
    """
    Returns the nth number in the fibonacci sequence
    """
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

print(fib(9))



In [48]:
instruction="1 ile 100 arasındaki 3 ve 5'e bölünen sayıları bulan python3 kodunu yaz"
CodeEdit(input=" ", instruction=instruction)

for i in range(1,101):
    if i%3==0 and i%5==0:
        print(i)



In [49]:
instruction="""Microsoft SQL tabloları:

Employee(id, name, department_id)
Department(id, name, address)
Sales_amount(id, employee_id, amount, date)

yukarıdaki tablolara göre son 5 ay içerisinde toplam olarak 10 bin tl satış yapan çalışanların isimlerini listeyen query'i yaz"""
CodeEdit(input=" ", instruction=instruction)

SELECT e.name
FROM Employee e
INNER JOIN Sales_amount s ON e.id = s.employee_id
WHERE s.date BETWEEN DATEADD(MONTH, -5, GETDATE()) AND GETDATE()
GROUP BY e.name
HAVING SUM(s.amount) >= 10000



In [50]:
input="""dogs = ["bill", "joe", "carl"]
car = []
dogs.forEach((dog) {
    car.push(dog);
});"""

instruction= "python'a dönüştür"

CodeEdit(input=input, instruction=instruction)

dogs = ["bill", "joe", "carl"];
car = [];
for dog in dogs:
    car.append(dog);



In [51]:
input = """
set.seed(42)
train_index <- createDataPartition(y, p = 0.9, list = FALSE)
X_train <- X[train_index, ]
X_test <- X[-train_index, ]
y_train <- y[train_index]
y_test <- y[-train_index]"""

instruction= "R'dan python'a dönüştür."

CodeEdit(input=input, instruction=instruction)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("data.csv")

X = df.drop(["y"], axis=1)
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)



In [ ]:
#

## Count of tokens for gpt-3.5-turbo-0301

In [52]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.5 MB/s eta 0:00:00


In [53]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
  """Returns the number of tokens used by a list of messages."""
  import tiktoken
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")

In [54]:
messages = [
  {"role": "system", "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English."},
  {"role": "system", "name":"example_user", "content": "New synergies will help drive top-line growth."},
  {"role": "system", "name": "example_assistant", "content": "Things working well together will increase revenue."},
  {"role": "system", "name":"example_user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
  {"role": "system", "name": "example_assistant", "content": "Let's talk later when we're less busy about how to do better."},
  {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
]

model = "gpt-3.5-turbo-0301"

print(f"{num_tokens_from_messages(messages, model)} prompt tokens counted.")
# Should show ~126 total_tokens

126 prompt tokens counted.
